In [15]:
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
import tensorflow as tf
import torch.nn.functional as F
import torchvision 
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import copy
import numpy as np
import matplotlib.pyplot as plt

In [16]:

transform = transforms.ToTensor()

batch_size = 64

train_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)


train_dataset_mnist = torch.utils.data.Subset(train_dataset_mnist,list(range(800)))

val_dataset_mnist = torch.utils.data.Subset(train_dataset_mnist,list(range(800, 1000)))


#train_dataset_mnist,val_dataset_mnist = random_split(train_dataset_mnist, [55000, 5000])
train_loader_mnist = torch.utils.data.DataLoader(train_dataset_mnist, batch_size=batch_size, shuffle=True, num_workers=2)

val_loader_mnist = torch.utils.data.DataLoader(val_dataset_mnist, batch_size=batch_size, shuffle=True, num_workers=2)
                                          
test_loader_mnist = torch.utils.data.DataLoader(test_dataset_mnist, batch_size=batch_size, shuffle=False, num_workers=2)

X_temp, y_temp = next(iter(train_loader_mnist))

print(X_temp[2], y_temp.shape)


tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [17]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)
#    return tf.random_normal(shape=size, stddev=xavier_stddev)

Discrimination
---------

In [31]:

D_W1 = xavier_init(size=[784, 128])
D_b1 = Variable(torch.zeros(128), requires_grad=True)

D_W2 = xavier_init(size=[128, 1])
D_b2 = Variable(torch.zeros(1), requires_grad=True)

theta_D = [D_W1, D_W2, D_b1, D_b2]

def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit


Generator
------------

In [32]:
G_W1 = xavier_init(size=[100, 128])
G_b1 = Variable(torch.zeros(128), requires_grad=True)

G_W2 = xavier_init(size=[128, 784])
G_b2 = Variable(torch.zeros(784), requires_grad=True)

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


Plotting
---------

In [33]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

Training
--------

In [34]:
G_sample = generator(Z)
G_solver = torch.optim.Adam(theta_G , lr=1e-3)
D_solver = optim.Adam(theta_D, lr=1e-3)


In [ ]:
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

In [ ]:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100